# Introduction
There are many ways data can end up with missing values. For example,
+ A 2 bedroom house won't include a value for the size of a third bedroom.
+ A survey respondent may choose not to share his income.

Most machine learning libraries (including skit-learn) give an error if the model using data with missing values. So, we need to deal with this. 

There're **3** main strategies :
1. Drop Columns with Missing Values
2. Imputation
3. An Extension To Imputation

# 1. A Simple Option: Drop Columns with Missing Values
The simplest option is to drop columns with missing values.
![Sample](https://storage.googleapis.com/kaggle-media/learn/images/Sax80za.png)
Unless most values in the dropped columns are missing, the model **loses access to a lot of (potentially useful!) information** with this approach

# 2. A Better Option: Imputation
**Imputation** fills in the missing value with some number. 

For instance, we can fill in the mean value along each column.
![](https://storage.googleapis.com/kaggle-media/learn/images/4BpnlPA.png)

The imputed value won't be exactly right in most cases, but it usually leads to more accurate models than just drop the column entirelly.

# 3. An Extension To Imputation
Imputation is the standard approach, and it usually works well. However, imputed values may be **systematically above or below their actual values** (which weren't collected in the dataset).

Or rows with missing values may be unique in some other way. In that case, your model would make better predictions by considering which values were originally missing.

![SAMPLE](https://storage.googleapis.com/kaggle-media/learn/images/UWOyg4a.png)

In this approach, we impute the missing values, as before. And, additionally, for each column with missing entries in the original dataset, we add a new column that shows the location of the imputed entries.

In some cases, this will meaningfully improve results. In other cases, it doesn't help at al


# Example

## Data making

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split

mel_data = pd.read_csv("./melb_data.csv")

y = mel_data['Price']

# Select only numerical features
X = mel_data.drop(['Price'],axis=1)
X = X.select_dtypes(exclude='object')

#Divide data
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,test_size=0.2,random_state=0)

# Calculate MAE each approach


In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train,X_test,y_train,y_test):
    model = RandomForestRegressor(n_estimators=100,random_state=0)
    model.fit(X_train,y_train)
    preds = model.predict(X_test)
    return mean_absolute_error(preds,y_test)

## Approach 1 : Drop Columns with Missing Values

In [15]:
#FIND
missing_cols = [col for col in X_train.columns
                if X_train[col].isnull().any()]

#DROP
dropped_X_train = X_train.drop(missing_cols,axis=1)
dropped_X_test = X_test.drop(missing_cols,axis=1)

print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(dropped_X_train,dropped_X_test,y_train,y_test))

MAE from Approach 1 (Drop columns with missing values):
175703.48185157913


## Approach 2 : Imputation

Next, we use `SimpleImputer` to replace missing values with the mean value along each column.

In [16]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_test = pd.DataFrame(my_imputer.transform(X_test))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_test.columns = X_test.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_test, y_train, y_test))
 

MAE from Approach 2 (Imputation):
169237.0268668034


We see that **Approach 2** has lower MAE than **Approach 1**, so **Approach 2** performed better on this dataset.

In [17]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_test.copy()

# Make new columns indicating what will be imputed
for col in missing_cols:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_test))

MAE from Approach 3 (An Extension to Imputation):
169795.45249719475


As we can see, **Approach 3** performed slightly worse than **Approach 2**.

## Why?????
So, why did imputation perform better than dropping the columns?
The training data has 10864 rows and 12 columns, where three columns contain missing data. For each column, **less than half** of the entries are missing. Thus, dropping the columns removes a lot of useful information, and so it makes sense that imputation would perform better.



In [20]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


# Conclusion
As is common, imputing missing values (in **Approach 2** and **Approach 3**) yielded better results, relative to when we simply dropped columns with missing values (in **Approach 1**).
